In [7]:
import random
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

## Ejercicio 7

Se propone simular la evolución de una epidemia utilizando un modelo basado en agentes, en el cual el desplazamiento se modela como random walking.

Se considera una población de N individuos (utilizar distintos valor de N) distribuidos uniformemente sobre un área de 100 m x 100 m

(Representar esta área como una grilla de celdas de 0,5 x 0,5 m)
Se considera que inicialmente el 2% de los individuos tiene la enfermedad, y existen 3 tipos e individuos:

| Tipo        | Porcentaje  | Caracteristicas
| ----------- | ----------- | -----------
| A.          | 70%         | Se desplaza una celda por instantes de tiempo
| B.          | 25%         | Se desplaza una celda por cada dos instantes de tiempo
| C.          | 5%          | Se desplaza una celda por cada cuatro instantes de tiempo

Esta enfermedad puede ser transmitida con una la siguiente probabilidad:


| Prob        | Distancia entre sano y enfermo 
| ----------- | ----------- 
| 0.5         | 3 < celdas <= 6        
| 0.7         | 3 celdas o menos       


Suponer que las personas contagiadas no se pueden curar.
Realice una simulación hasta 5000 instantes de tiempo, o hasta que la población completa se contagie.

* Una animación de la situación planteada
* Obtenga estadísticas de los tiempos que demoran los individuos en contagiarse. Grafique.
* Graficar la curva de evolución de la epidemia midiendo.
   *  Cantidad de personas sanas en función del tiempo
   * Cantidad de personas enfermas en función del tiempo

In [8]:
TAMANIO_MATRIZ = 100
MOVIMIENTOS_POSIBLES = [(0,-1), (0,1), (1,0), (-1,0)]

class Individuo: 
    def __init__(self, reset_contador, x, y): 
        self.reset_contador = reset_contador
        self.contador = reset_contador
        self.x = x
        self.y = y
        self.infectado = False
        
        
    def obtener_posicion(self):
        return self.x,self.y
    
    def infectar(self):
        self.infectado = True
        
    def mover(self):
        self.contador -= 1
        if self.contador == 0:
            mov_x, mov_y = random.choice(MOVIMIENTOS_POSIBLES)
            self.x = (self.x + mov_x) % TAMANIO_MATRIZ
            self.y = (self.y + mov_y) % TAMANIO_MATRIZ
            self.contador = self.reset_contador

In [ ]:
CANTIDAD_PERSONAS = 100

personas = []
for i in range(CANTIDAD_PERSONAS):
    rand_x = int(random.uniform(0,TAMANIO_MATRIZ))
    rand_y = int(random.uniform(0,TAMANIO_MATRIZ))
    tipo = random.uniform(0,1)
    reset_counter = 2
    if tipo <= 0.7:
        reset_counter = 1
    if tipo >= 0.95:
        reset_counter = 4
    
    individuo = Individuo(reset_counter, rand_x, rand_y)
    personas.append(individuo)

random.shuffle(personas)
for i in range(int(CANTIDAD_PERSONAS*0.02)):
    personas[i].infectar()
    
fig = plt.figure(figsize=(7,7))
ax = plt.axes(xlim=(-1,TAMANIO_MATRIZ+1),ylim=(-1,TAMANIO_MATRIZ+1))

posiciones_x = []
posiciones_y = []
colores = []
for p in personas:
    posiciones_x.append(p.obtener_posicion()[0])
    posiciones_y.append(p.obtener_posicion()[1])
    if p.infectado:
        colores.append('red')
    else:
        colores.append('blue')

scatter = ax.scatter(posiciones_x, posiciones_y, color=colores)

def update(frame_number):
    posiciones = []
    colores = []
    for p in personas:
        p.mover()
        posiciones.append(p.obtener_posicion())
        if p.infectado:
            colores.append('red')
        else:
            colores.append('blue')
    scatter.set_offsets(posiciones)
    return scatter,

anim = FuncAnimation(fig, update, interval=10, frames=5000)
HTML(anim.to_html5_video())